# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
X = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type", "hardship_flag", "debt_settlement_flag"],drop_first=True)
X.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,0,0,0,0,1,1,1,0


## Split the Data into Training and Testing

In [6]:

# Create our features
X = pd.get_dummies(df.drop(columns="loan_status",axis=1),drop_first=True)

# Create our target
y = df.loan_status


In [7]:

X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,0,0,0,1,1,1,0


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y
)

X_train.shape

(51612, 86)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
from sklearn.preprocessing import StandardScaler
# Create the StandardScaler instance

scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data

X_train_scaler = X_scaler.transform(X_train)
X_test_scaler = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier
# Resample the training data with the BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
# Calculated the balanced accuracy score

prediction_rf = brfc.predict(X_test)
balanced_accuracy_score(y_test, prediction_rf)

0.795829959187949

In [15]:
from sklearn.metrics import confusion_matrix
# Display the confusion matrix

confusion_matrix(y_test, prediction_rf)


array([[   62,    25],
       [ 2071, 15047]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_rf, digits=3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk      0.029     0.713     0.879     0.056     0.791     0.616        87
   low_risk      0.998     0.879     0.713     0.935     0.791     0.637     17118

avg / total      0.993     0.878     0.713     0.930     0.791     0.637     17205



In [17]:
# List the features sorted in descending order by feature importance
features_level = pd.DataFrame(
    brfc.feature_importances_,
    index=X_train.columns,
    columns = ["features lvl"])
    
features_level.sort_values("features lvl", ascending=False)

,features lvl
total_rec_prncp,0.078795
total_rec_int,0.068296
total_pymnt_inv,0.063323
total_pymnt,0.060703
last_pymnt_amnt,0.048325
...,...
num_tl_120dpd_2m,0.000000
collection_recovery_fee,0.000000
num_tl_30dpd,0.000000
delinq_amnt,0.000000


### Easy Ensemble Classifier

In [18]:
from imblearn.ensemble import EasyEnsembleClassifier
# Train the Classifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [19]:
# Calculated the balanced accuracy score

prediction_eec = eec.predict(X_test)
balanced_accuracy_score(y_test, prediction_eec)

0.9263912558266958

In [20]:
# Display the confusion matrix

confusion_matrix(y_test, prediction_eec)

array([[   79,     8],
       [  946, 16172]])

In [21]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, prediction_eec, digits=3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk      0.077     0.908     0.945     0.142     0.926     0.855        87
   low_risk      1.000     0.945     0.908     0.971     0.926     0.861     17118

avg / total      0.995     0.945     0.908     0.967     0.926     0.861     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    - Random Forest accuracy score ~ 0.79
    - Easy ensemble accuracy score ~ 0.93
    - Easy ensemble has the best balanced accuracy score.

2. Which model had the best recall score?

    - Random Forest recall score ~ 0.91
    - Easy ensemble recall score ~ 0.94
    - Easy ensemble has the best recall score.

3. Which model had the best geometric mean score?

    - Random Forest geometric mean score ~ 0.78
    - Easy ensemble geometric mean score ~ 0.93
    - Easy ensemble has the best geometric mean score.

4. What are the top three features

In [22]:
top_three_features = features_level.nlargest(3, "features lvl")
top_three_features

,features lvl
total_rec_prncp,0.078795
total_rec_int,0.068296
total_pymnt_inv,0.063323
